# Heat transfer 2D

Slove heat transfer by hand-mesh using FEM.

## Module

In [1]:
import numpy as np


class Boundary:
    def __init__(self, node):
        self.node = node

    def getMat(self):
        pass

    def getLoad(self):
        pass

    def getBoundOn(self, ele):
        if self.node[0] in ele.node and self.node[1] in ele.node:
            a, b = ele.node.index(self.node[0]), ele.node.index(self.node[1])
            if b < a:
                return [b, a]
            return [a, b]
        return None


class Convection(Boundary):
    def __init__(self, n1, n2, h, Tf):
        super().__init__([n1, n2])
        self.h = h
        self.Tf = Tf

    def getMat(self):
        return self.h * getLen(*self.node) * np.matrix("2 1;1 2") / 6

    def getLoad(self):
        return self.h * getLen(*self.node) * self.Tf / 2 * np.array([1, 1])


class Element:
    def __init__(self, node):
        self.node = node
        self.load = np.zeros(len(node))
        self.matrix = np.zeros([len(node), len(node)])

    def writeGlobal(self, mat, load):
        mat[np.ix_(self.node, self.node)] += self.matrix
        load[self.node] += self.load

    def setBound(self, bound):
        ind = bound.getBoundOn(self)
        self.matrix[np.ix_(ind, ind)] += bound.getMat()
        self.load[ind] = bound.getLoad()


class Square(Element):
    def __init__(self, n1, n2, n3, n4, kx, ky):
        """
        Define the square
        n4 n3
        n1 n2
        """
        super().__init__([n1, n2, n3, n4])

        # prepare matrix
        a = np.array([2, -2, -1, 1])
        mx = np.stack([a, -a, -np.flip(a), np.flip(a)])
        my = mx.copy()
        my[mx ==  1] = -2
        my[mx == -2] =  1

        # get length of rectangle
        l12 = getLen(n1, n2)
        l23 = getLen(n2, n3)

        # calculate condutance
        self.matrix = (kx * l23 / l12 * mx + ky * l12 / l23 * my) / 6


class Triangle(Element):
    def __init__(self, n1, n2, n3, kx, ky):
        """
        Define the triangle
            n3
        n1     n2
        """
        super().__init__([n1, n2, n3])

        # prepare matrix
        pos = nodes[self.node]
        dpos = np.roll(pos, -1, axis=0) - np.roll(pos, -2, axis=0)
        self.matrix = kx * np.outer(dpos[:, 0], dpos[:, 0]) \
                    + ky * np.outer(dpos[:, 1], dpos[:, 1])
        A = np.cross(dpos[0], dpos[1]) / 2
        self.matrix = self.matrix / 4 / A


def getLen(n1, n2):
    return np.sqrt(((nodes[n1] - nodes[n2]) ** 2).sum())

## Custom data
* nodes: position of nodes, is a list of (x, y)
* elements: One of Square and Triangle
    * Square parameters (node_i, node_j, node_m, node_n, kx, ky)
    * Triangle parameters (node_i, node_j, node_k, kx, ky)
* boundarys: Convection
    * Convection parameters (node_i, node_j, h, T_env)
* Const:
    * Each tuple in the list has two values(node_i, T_i),
      which means T(node_i) = T_i

In [2]:
nodes = [[0, 0],
    [0, 0], [1, 0],
    [0, 1], [1, 1], [2, 1],
    [0, 2], [1, 2], [2, 2], [3, 2]
]
nodes = np.array(nodes) * .1

elements = [
    Square(1, 2, 4, 3, 1.4, 1.4),
    Square(3, 4, 7, 6, 1.4, 1.4),
    Square(4, 5, 8, 7, 1.4, 1.4),
    Triangle(2, 5, 4, 1.4, 1.4),
    Triangle(5, 9, 8, 1.4, 1.4)
]

boundarys = [
    Convection(6, 7, 20, 30),
    Convection(7, 8, 20, 30),
    Convection(8, 9, 20, 30),
]

Const = [
    [1, 100],
    [2, 100]
]

np.set_printoptions(3)

## solve it

In [3]:
n = len(nodes)
T = np.zeros([n, n])
L = np.zeros([n])

for ele in elements:
    print("ele")
    print(ele.matrix)
    for bound in boundarys:
        if bound.getBoundOn(ele):
            ele.setBound(bound)
            print("bound")
            print(ele.matrix)
    ele.writeGlobal(T, L)

print("Table Load")
print(T[1:, 1:])
print(L[1:])

for c in Const:
    T[c[0], :] = 0
    T[c[0], c[0]] = 1
    L[c[0]] = c[1]

print("Table Load with Boundary")
print(T[1:, 1:])
print(L[1:])

result = np.linalg.inv(T[1:, 1:]).dot(L[1:])
print("Result")
print(result)

ele
[[ 0.933 -0.233 -0.467 -0.233]
 [-0.233  0.933 -0.233 -0.467]
 [-0.467 -0.233  0.933 -0.233]
 [-0.233 -0.467 -0.233  0.933]]
ele
[[ 0.933 -0.233 -0.467 -0.233]
 [-0.233  0.933 -0.233 -0.467]
 [-0.467 -0.233  0.933 -0.233]
 [-0.233 -0.467 -0.233  0.933]]
bound
[[ 0.933 -0.233 -0.467 -0.233]
 [-0.233  0.933 -0.233 -0.467]
 [-0.467 -0.233  1.6    0.1  ]
 [-0.233 -0.467  0.1    1.6  ]]
ele
[[ 0.933 -0.233 -0.467 -0.233]
 [-0.233  0.933 -0.233 -0.467]
 [-0.467 -0.233  0.933 -0.233]
 [-0.233 -0.467 -0.233  0.933]]
bound
[[ 0.933 -0.233 -0.467 -0.233]
 [-0.233  0.933 -0.233 -0.467]
 [-0.467 -0.233  1.6    0.1  ]
 [-0.233 -0.467  0.1    1.6  ]]
ele
[[ 0.7  0.  -0.7]
 [ 0.   0.7 -0.7]
 [-0.7 -0.7  1.4]]
ele
[[ 0.7  0.  -0.7]
 [ 0.   0.7 -0.7]
 [-0.7 -0.7  1.4]]
bound
[[ 0.7    0.    -0.7  ]
 [ 0.     1.367 -0.367]
 [-0.7   -0.367  2.067]]
Table Load
[[ 0.933 -0.233 -0.233 -0.467  0.     0.     0.     0.     0.   ]
 [-0.233  1.633 -0.467 -0.933  0.     0.     0.     0.     0.   ]
 [-0.233 -0